# Appendix - LSTM + Attention from Scratch

In case any students want to deeply understand LSTM.  You can take a look at this.  

We shall explore the following

    1. LSTM and its variants
        - Vanilla LSTM
        - Coupled Gate LSTM
        - Peephole LSTM
        - BiLSTM
    2. Attention Mechanism
        - General Attention
        - Self-Attention
        
To understand, we code from scratch using pytorch building units.

Note that in reality, we don't do from scratch, because the built-in pytorch LSTM has some cudnn optimization that makes it faster than ours.

In [1]:
import torchtext
import torch
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
#some hyperparameters
input_dim = 5000  #just for example
hidden_dim = 256
embed_dim = 300
output_dim = 1

batch_size = 32

## 1. LSTM and its variants

We have learned in class that LSTM was designed to avoid the long term dependency problem as well as to helps with the problem of vanishing and exploding gradients.

The key to LSTM is the cell state and the 3 gates to 'protect' and 'control' the cell states.

We will now look in to the components inside and implement them line by line :)

**The expected shape of LSTM input is SHAPE : (bs, seq_len, input_dim)**

For **EACH** time step of our sequence, these are the operations inside LSTM cell.

The first step in our LSTM is to decide what information we’re going to throw away from the previous cell state. This decision is made by a sigmoid layer called the “forget gate layer.” It looks at $\mathbf{h}_{t-1}$ and $\mathbf{x}_t$, and outputs a number between 0 and 1. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.”

$\mathbf{f}_t = \sigma(\mathbf{W}_f \ \mathbf{h}_{t-1} + \mathbf{U}_f \ \mathbf{x}_t + \mathbf{b}_f)$

The next step is to decide what new information we’re going to store in the cell state. This has two parts.
First, a sigmoid layer called the “input gate layer” decides which values we’ll update.

$\mathbf{i}_t = \sigma(\mathbf{W}_i \ \mathbf{h}_{t-1} + \mathbf{U}_i \ \mathbf{x}_t + \mathbf{b}_i)$

Next, a tanh layer creates a vector of new 'candidate' values, $\tilde{\mathbf{c}}_t$ (aka. $\mathbf{g}_t$), that could be added to the state. In the next step, we’ll combine these two to create an update to the state.

$\mathbf{g}_t = \mathbf{tanh} \ (\mathbf{W}_g \ \mathbf{h}_{t-1} + \mathbf{U}_g \ \mathbf{x}_t  + \mathbf{b}_g)$

It’s now time to update the old cell state, $\mathbf{c}_{t-1}$, into the new cell state $\mathbf{c}_t$. The previous steps already decided what to do, we just need to actually do it.
We multiply the old state by f_t, forgetting the things we decided to forget earlier.
Then we add $\mathbf{i}_t \circ \mathbf{g}_t$. This is the new candidate values, scaled by how much we decided to update each state value.

$\mathbf{c}_t = \mathbf{f}_t \circ \mathbf{c}_{t-1} + \mathbf{i}_t \circ \mathbf{g}_t$

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.

$\mathbf{o}_t = \sigma( \mathbf{W}_o \ \mathbf{h}_{t-1} + \mathbf{U}_o \ \mathbf{x}_t + \mathbf{b}_o)$

$\mathbf{h}_t = \mathbf{o}_t \circ \tanh \ (\mathbf{c}_t)$

In conclusion, these are the formula that we need to implement in our LSTM_cell class :

$\mathbf{f}_t = \sigma(\mathbf{W}_f \ \mathbf{h}_{t-1} + \mathbf{U}_f \ \mathbf{x}_t + \mathbf{b}_f)$

$\mathbf{i}_t = \sigma(\mathbf{W}_i \ \mathbf{h}_{t-1} + \mathbf{U}_i \ \mathbf{x}_t + \mathbf{b}_i)$

$\mathbf{o}_t = \sigma( \mathbf{W}_o \ \mathbf{h}_{t-1} + \mathbf{U}_o \ \mathbf{x}_t + \mathbf{b}_o)$

$\mathbf{g}_t = \tanh \ (\mathbf{W}_g \ \mathbf{h}_{t-1} + \mathbf{U}_g \ \mathbf{x}_t  + \mathbf{b}_g)$

$\mathbf{c}_t = \mathbf{f}_t \circ \mathbf{c}_{t-1} + \mathbf{i}_t \circ \mathbf{g}_t$

$\mathbf{h}_t = \mathbf{o}_t \circ \tanh \ (\mathbf{c}_t)$

where

$\mathbf{h}_{t-1}$ is the hidden state from the previous time step [SHAPE : (bs, hidden_dim)] << no seq_len because it is only at time step t-1

$\mathbf{x}_t$ is the input of the current time step [SHAPE : (bs, hidden_dim)] << no seq_len because it is only at time step t

$\mathbf{W}$ are the weights that would be multiply with the hidden states [SHAPE : (hidden_dim, hidden_dim)]

$\mathbf{U}$ are the weights that would be multiply with the inputs [SHAPE : (input_dim, hidden_dim)]

$\mathbf{b}$ are the biases that would be added to the values before they are passed to sigmoid or tanh [SHAPE : (hidden_dim)]

$\circ$ is the Hadamard product as known as element-wise multiplication

** $\tilde{\mathbf{c}}_t$ and $\mathbf{g}_t$ can be used interchangeably

### 1.1 Vanilla LSTM from Scratch

In [3]:
import math
class LSTM_cell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        
        # initialise the trainable Parameters
        self.U_i = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_i = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_i = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.U_f = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_f = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_f = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.U_g = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_g = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_g = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.U_o = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_o = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_o = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.init_weights()
    
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
    def forward(self, x, init_states=None):
        """
        x.shape =  (bs, seq_len, input_dim)
        """
        bs, seq_len, _ = x.shape
        output = []
        
        # initialize the hidden state and cell state for the first time step 
        if init_states is None:
            h_t  = torch.zeros(bs, self.hidden_dim).to(x.device)
            c_t  = torch.zeros(bs, self.hidden_dim).to(x.device)
        else:
            h_t, c_t = init_states
        
        # For each time step of the input x, do ...
        for t in range(seq_len):
            x_t = x[:, t, :] # get x data of time step t (SHAPE: (batch_size, input_dim))
            
            f_t = torch.sigmoid(    h_t @ self.W_f  +  x_t @ self.U_f  +  self.b_f)
            i_t = torch.sigmoid(    h_t @ self.W_i  +  x_t @ self.U_i  +  self.b_i)
            o_t = torch.sigmoid(    h_t @ self.W_o  +  x_t @ self.U_o  +  self.b_o)
            g_t = torch.tanh(       h_t @ self.W_g  +  x_t @ self.U_g   + self.b_g)
            c_t = (f_t * c_t) + (i_t * g_t)
            h_t = o_t * torch.tanh(c_t)
            
            output.append(h_t.unsqueeze(0)) # reshape h_t to (1, batch_size, hidden_dim), then append to the list of hidden states

        output = torch.cat(output, dim = 0) # concatenate h_t of all time steps into SHAPE :(seq_len, batch_size, hidden_dim)
        output = output.transpose(0, 1).contiguous() # just transpose to SHAPE :(batch_size, seq_len, hidden_dim)
        return output, (h_t, c_t)

### Run this cell to check LSTM Cell can run

In [4]:
my_LSTM_cell = LSTM_cell(embed_dim, hidden_dim).to(device)

test_data = torch.ones((batch_size, 100, embed_dim)).to(device)
output, (h_t, c_t) = my_LSTM_cell(test_data)

assert output.shape == torch.Size([32, 100, 256])
assert h_t.shape    == torch.Size([32, 256])
assert c_t.shape    == torch.Size([32, 256])

Many variants of LSTM have been developed which are slightly different from Vanilla/Basic LSTM that we have just implemented above

### 1.2 Peephole LSTM
One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “Peephole Connections.” This means that we let all the gate layers look at the cell state.

$\mathbf{f}_t = \sigma(\mathbf{W}_f \ \mathbf{h}_{t-1} + \mathbf{U}_f \ \mathbf{x}_t + \mathbf{P}_f \ \mathbf{c}_t + \mathbf{b}_f)$

$\mathbf{i}_t = \sigma(\mathbf{W}_i \ \mathbf{h}_{t-1} + \mathbf{U}_i \ \mathbf{x}_t + \mathbf{P}_i \ \mathbf{c}_t + \mathbf{b}_i)$

$\mathbf{o}_t = \sigma( \mathbf{W}_o \ \mathbf{h}_{t-1} + \mathbf{U}_o \ \mathbf{x}_t + \mathbf{P}_o \ \mathbf{c}_t + \mathbf{b}_o)$

$\mathbf{g}_t = \text{tanh} \ (\mathbf{W}_g \ \mathbf{h}_{t-1} + \mathbf{U}_g \ \mathbf{x}_t  + \mathbf{b}_g)$

$\mathbf{c}_t = \mathbf{f}_t \circ \mathbf{c}_{t-1} + \mathbf{i}_t \circ \mathbf{g}_t$

$\mathbf{h}_t = \mathbf{o}_t \circ \tanh \ (\mathbf{c}_t)$

We can see that every gate now has $\mathbf{c}_t$ as their input. And we also have 3 new parameters; $\mathbf{P}_f$, $\mathbf{P}_i$ and $\mathbf{P}_o$ which has the same shape as $\mathbf{W}$.

### 1.3 Coupled LSTM

Another variation is to use Coupled forget and input gates. Instead of separately deciding what to forget and what we should add new information to, we make those decisions together. We only forget when we’re going to input something in its place. We only input new values to the state when we forget something older. The difference is very simple. The input gate is now $(1 - \mathbf{f}_t)$


$\mathbf{f}_t = \sigma(\mathbf{W}_f \ \mathbf{h}_{t-1} + \mathbf{U}_f \ \mathbf{x}_t + \mathbf{b}_f)$

$\mathbf{i}_t = (1 - \mathbf{f}_t)$

$\mathbf{o}_t = \sigma( \mathbf{W}_o \ \mathbf{h}_{t-1} + \mathbf{U}_o \ \mathbf{x}_t + \mathbf{b}_o)$

$\mathbf{g}_t = \text{tanh} \ (\mathbf{W}_g \ \mathbf{h}_{t-1} + \mathbf{U}_g \ \mathbf{x}_t + \mathbf{b}_g)$

$\mathbf{c}_t = \mathbf{f}_t \circ \mathbf{c}_{t-1} + \mathbf{i}_t \circ \mathbf{g}_t$

$\mathbf{h}_t = \mathbf{o}_t \circ \text{tanh} \ (\mathbf{c}_t)$

### Vanilla / Peephole / Coupled LSTM from Scratch

In [5]:
class new_LSTM_cell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, lstm_type: str):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.lstm_type = lstm_type
        
        # initialise the trainable Parameters
        self.U_i = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_i = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_i = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.U_f = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_f = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_f = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.U_g = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_g = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_g = nn.Parameter(torch.Tensor(hidden_dim))
        
        self.U_o = nn.Parameter(torch.Tensor(input_dim, hidden_dim))
        self.W_o = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.b_o = nn.Parameter(torch.Tensor(hidden_dim))
        
        if self.lstm_type == 'peephole' :
            self.P_i = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
            self.P_f = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
            self.P_o = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
            
        self.init_weights()

    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
    def forward(self, x, init_states=None):
        bs, seq_len, _ = x.shape
        output = []
        
        # initialize the hidden state and cell state for the first time step 
        if init_states is None:
            h_t  = torch.zeros(bs, self.hidden_dim).to(x.device)
            c_t  = torch.zeros(bs, self.hidden_dim).to(x.device)
        else:
            h_t, c_t = init_states
        
        # For each time step of the input x, do ...
        for t in range(seq_len):
            x_t = x[:, t, :] # get x data of time step t (SHAPE: (batch_size, input_dim))
            
            if self.lstm_type in ['vanilla', 'coupled'] :
                f_t = torch.sigmoid(    h_t @ self.W_f  +  x_t @ self.U_f  +  self.b_f)
                o_t = torch.sigmoid(    h_t @ self.W_o  +  x_t @ self.U_o  +  self.b_o)
                if self.lstm_type == 'vanilla':
                    i_t = torch.sigmoid(    h_t @ self.W_i  +  x_t @ self.U_i  +  self.b_i)
                if self.lstm_type == 'coupled':
                    i_t = (1 - f_t)
            if self.lstm_type == 'peephole' :
                i_t = torch.sigmoid( h_t @ self.W_i + x_t @ self.U_i + c_t @ self.P_i + self.b_i) # SHAPE: (batch_size, hidden_dim)
                f_t = torch.sigmoid( h_t @ self.W_f + x_t @ self.U_f + c_t @ self.P_f + self.b_f) # SHAPE: (batch_size, hidden_dim)
                o_t = torch.sigmoid( h_t @ self.W_o + x_t @ self.U_o + c_t @ self.P_o + self.b_o) # SHAPE: (batch_size, hidden_dim)
            
            g_t = torch.tanh(       h_t @ self.W_g  +  x_t @ self.U_g   + self.b_g)
            c_t = (f_t * c_t) + (i_t * g_t)
            h_t = o_t * torch.tanh(c_t)
            
            output.append(h_t.unsqueeze(0)) # reshape h_t to (1, batch_size, hidden_dim), then append to the list of hidden states

        output = torch.cat(output, dim = 0) # concatenate h_t of all time steps into SHAPE :(seq_len, batch_size, hidden_dim)
        output = output.transpose(0, 1).contiguous() # just transpose to SHAPE :(seq_len, batch_size, hidden_dim)
        return output, (h_t, c_t)

### Run this cell to check if all types of your LSTM Cells can run

In [6]:
Vanilla_LSTM_cell = new_LSTM_cell(embed_dim, hidden_dim, lstm_type = 'vanilla').to(device)
test_data = torch.ones((batch_size, 100, embed_dim)).to(device)
output, (h_t, c_t) = Vanilla_LSTM_cell(test_data)
assert output.shape == torch.Size([32, 100, 256])
assert h_t.shape    == torch.Size([32, 256])
assert c_t.shape    == torch.Size([32, 256])

Coupled_LSTM_cell = new_LSTM_cell(embed_dim, hidden_dim, lstm_type = 'coupled').to(device)
test_data = torch.ones((batch_size, 100, embed_dim)).to(device)
output, (h_t, c_t) = Coupled_LSTM_cell(test_data)
assert output.shape == torch.Size([32, 100, 256])
assert h_t.shape    == torch.Size([32, 256])
assert c_t.shape    == torch.Size([32, 256])

Peephole_LSTM_cell = new_LSTM_cell(embed_dim, hidden_dim, lstm_type = 'peephole').to(device)
test_data = torch.ones((batch_size, 100, embed_dim)).to(device)
output, (h_t, c_t) = Peephole_LSTM_cell(test_data)
assert output.shape == torch.Size([32, 100, 256])
assert h_t.shape    == torch.Size([32, 256])
assert c_t.shape    == torch.Size([32, 256])

### 1.4 BiLSTM model for sequence classification

We now have the basic variants of LSTM cells. But what about Bidirectional LSTM. How do we implement that?

The answer is simple. We create **2 LSTM cells** then pass our normal input to one of them, and pass the **flipped** input to the other. (reverse the order of sequence)

Then we take the last hidden state from the 2 LSTM (one would be the hidden state at the last word of the sentence and another at the first word of the sentence) and concatenate them. Like this we have information of the sequence from both directions!

Formally these are the formula

$\overrightarrow{\mathbf{h}}_t = LSTM(\mathbf{x}_t, \overrightarrow{\mathbf{h}}_{t-1})$

$\overleftarrow{\mathbf{h}}_t = LSTM(\mathbf{x}_t, \overleftarrow{\mathbf{h}}_{t+1})$

$\mathbf{h}_t = \sigma(\mathbf{W}_y[\overrightarrow{\mathbf{h}}_t ; \overleftarrow{\mathbf{h}}_{t}] + \mathbf{b}_y )$

Then we should pass $\mathbf{h}_t$ to another Linear Layer to get the output for binary classification.

### biLSTM from Scratch

**( Let's use our 'vanilla' LSTM_cell)**

In [7]:
class BiLSTM_model(nn.Module):
    def __init__(self, input_dim: int, embed_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()
        self.num_directions = 2
        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=pad_idx)
        self.hidden_dim = hidden_dim
        
        self.forward_lstm   =  new_LSTM_cell(embed_dim, hidden_dim, lstm_type = 'vanilla')
        self.backward_lstm  =  new_LSTM_cell(embed_dim, hidden_dim, lstm_type = 'vanilla')
        
        # These should be torch Parameters
        self.W_h = nn.Parameter(torch.Tensor(hidden_dim*self.num_directions, hidden_dim*self.num_directions ))
        self.b_h = nn.Parameter(torch.Tensor(hidden_dim*self.num_directions))
        
        self.fc  = nn.Linear(hidden_dim*self.num_directions, output_dim)
    
        self.init_weights()
    
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
    def forward(self, text, text_lengths):
        embedded      = self.embedding(text)
        embedded_flip =  torch.flip(embedded, [1]) 
        
        output_forward, (hn_forward, cn_forward)    = self.forward_lstm(embedded, init_states=None)
        output_backward, (hn_backward, cn_backward) = self.backward_lstm(embedded_flip, init_states=None)
        
        concat_hn = torch.cat( (hn_forward, hn_backward), dim=1 ) 
        ht        = torch.sigmoid( concat_hn @ self.W_h + self.b_h)

        return self.fc(ht)

## 2. Attention Mechanism
The attention mechanism was first born to help memorize long source sentences in neural machine translation (NMT). Rather than building a single context vector out of the encoder’s last hidden state, the attention mechanism creates shortcuts between the context vector and the entire source input. The weights of these shortcut connections are customizable for each output element.
While the context vector has access to the entire input sequence, we don’t need to worry about forgetting. The alignment between the source and target is learned and controlled by the context vector. Essentially the context vector consumes three pieces of information:

    - encoder hidden states
    - decoder hidden states
    - alignment between source and target
    
This is the same mechanism that we have learned in class, which is actually called 'Cross Attention'.

However, in this assignment we are making a classification model so we only have the encoder hidden states and our target would be the class decision.

### 2.1 General Attention Mechanism

First, we will be creating an LSTM + General Attention model for classification.

Our General Attention mechanism is going to capture how the last encoder hidden state (aka. the 'queries') 'relates' to the other hidden states in the sequence (a.k.a. the 'keys'). ( how much our classification decision is related to each of the hidden states)
Then we will scale the output (a.k.a. the 'values') according to the Attention Weights (computed from the Alignment Scores), in order to retain focus on words that are relevant to the query. In doing so, it produces an attention output that we will input to a fully connected layer for the result of our classification task.


**These are the steps we need to implement :**

We will pass our data through LSTM first then pass the outputs of LSTM to the General Attention mechanism.
    
**1. Get the components we need for our Attention Mechanism ('query', 'keys' and 'values')**
- Get the last encoder hidden states ($\mathbf{h}_N$) = last hidden state of last LSTM layer
        - Hint : can be found in 'hn'
        - Should be of shape [bs, hidden dim * num_directions]
        - a.k.a. 'query'
- Get the hidden states of every time step from the last layer of LSTM ($\mathbf{H}$)
        - Hint : can be found in 'output' 
        - Should be of shape  [bs, seq len, hidden_dim * num_directions]
        - a.k.a. 'keys'
        - This will be matched with our h_t to get the Attention Scores.
- Get the hidden states of every time step from the last layer of LSTM ($\mathbf{H}$)
        - Hint : can be found in 'output' 
        - Should be of shape  [bs, seq len, hidden_dim * num_directions]
        - a.k.a. 'values'
        - This will be weighted by Attention Weights to get the Context.
** In our case, we are implementing Attention in Classification model so our 'keys' and 'values' are the same thing

**2.  Calculate Alignment Scores:**

Calculate the Alignment Scores by matching the **'query'** with each of the **'keys'**. This matching operation is computed as the **dot product** of our specific 'query' with each of the hidden states or the 'key' vector. This is to get the scores of how 'related' the 'query' is to each 'key' or each hidden state.

$\mathbf{e}_t = [\mathbf{h}_N^T \ \mathbf{h}_1,  \mathbf{h}_N^T \ \mathbf{h}_2,  ..., \mathbf{h}_N^T \ \mathbf{h}_N] \in \mathbb{R}^N $

where

$ \mathbf{h}_1, ..., \mathbf{h}_N \in \mathbb{R}^h ; \mathbf{H} \in \mathbb{R}^{N,h}$

Hint : We can multiply our 'query' with all of the 'keys' at once by using the matrix form of the 'query' ($\mathbf{H}$) ( we have to keep shape of batch size at the first dimension so **torch.bmm** might come in handy !)

Hint2: Alignment Scores should be of shape :  [batch_size, seq_len, 1]

**3. Calculate Attention Weights :**

We pass the Alignment Scores through a **softmax** operation to convert the scores into probabilities called the 'Attention Weights'
This method is called **soft-attention** which help make the model smooth and differentiable.

$\alpha_t = \text{softmax}(\mathbf{e}_t) \in \mathbb{R}^N$

Hint : our softmaxed Attention Weights should still have the same shape as Alignment Score

**4. Calculate Context Vector :**

Use the Attention Weight to scale the output **'values'** to get the 'context vector'. In this example, the 'values' is the same as the 'keys' which is the hidden states of every time step from the last layer of LSTM. A context vector is a **weighted sum** of the value vectors, V_ki.

$ \mathbf{c}_t = \mathbf{H}^T \ \alpha_t \in \mathbb{R}^h $

Hint : Again, we can use the matrix form to get the weighted sum in one operation.
The resulting context should be of shape [bs, hidden_size * num_directions]

**5. Finally, we use this Context Vector as the output of our Attention Mechanism**


### LSTM + General Attention

In [8]:
import torch.nn as nn
from torch.nn import functional as F

class LSTM_GAtt(nn.Module):
    def __init__(self, input_dim: int, embed_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=pad_idx)
        
        # let's use pytorch's LSTM
        self.lstm = nn.LSTM(embed_dim, 
                           hidden_dim, 
                           num_layers=num_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        # Linear Layer for binary classification 
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def attention_net(self, lstm_output, hn):
        
        h_t      = hn.unsqueeze(2)
        H_keys   = torch.clone(lstm_output)
        H_values = torch.clone(lstm_output)
        
        alignment_score   = torch.bmm(H_keys, h_t).squeeze(2) # SHAPE : (bs, seq_len, 1)
        
        soft_attn_weights = F.softmax(alignment_score, 1) # SHAPE : (bs, seq_len, 1)
        
        context           = torch.bmm(H_values.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2) # SHAPE : (bs, hidden_size * num_directions)
        
        return context

    def forward(self, text, text_lengths):

        embedded = self.embedding(text) # SHAPE : (batch_size, seq_len, embed_dim)

        lstm_output, (hn, cn) = self.lstm(embedded)
        
        # This is how we concatenate the forward hidden and backward hidden from Pytorch's BiLSTM
        hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)

        attn_output = self.attention_net(lstm_output, hn)
        
        return self.fc(attn_output)

### 2.2 Self Attention Mechanism

Self-attention, also known as intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the same sequence.
The self-attention mechanism allows the inputs to interact with each other (“self”) and find out who they should pay more attention to (“attention”). The outputs are aggregates of these interactions and attention scores.

It has been shown to be very useful in machine reading, abstractive summarization, or image description generation.

You might have noticed from the previous part that the are 3 main vector/matrix in the attention mechanism, which are 'queries', 'keys' and 'value'.
Self-attention also need the same elements but we only have 'self' for the model to consider so 'queries', 'keys' and 'value' is all made from our input. Other steps are very similar to General Attention.

Same with the previous part, we will pass our data through LSTM first then pass the outputs of LSTM to the Self Attention mechanism.

**1. Get the components we need for our Attention Mechanism**

Make **3 copies** of $ \mathbf{H} $ (hidden states of every time step from the last layer of LSTM)

        - Hint : can be found in 'output' 
        - Should be of shape  [bs, seq len, hidden_dim * num_directions]

**2. Initialize 3 Linear Layers :**

    - Initialize 3 Linear Layer called 'lin_Q', 'lin_K', 'lin_V'
    
    - input_dim = hidden_dim * num_direction
    - output_dim = hidden_dim * num_direction
    
**3. Pass each copy of lstm_output through each of the Linear Layer.**
    
Pass each copy of $\mathbf{H}$ through each of the Linear Layer so that we have learnable weights for generating the queries, keys and values

$ \mathbf{Q} = \mathbf{H}^T \ \mathbf{W}_q + \mathbf{b}_q \in \mathbb{R}^{N,h} $

$ \mathbf{K} = \mathbf{H}^T \ \mathbf{W}_k + \mathbf{b}_k \in \mathbb{R}^{N,h} $

$ \mathbf{V} = \mathbf{H}^T \ \mathbf{W}_v + \mathbf{b}_v \in \mathbb{R}^{N,h} $

*Hint: Expected SHAPE : (bs, seq_len, n_hidden * num_directions)

**4. Calculate Alignment Scores:**

    - Matching the 'query' with the 'keys'.
    
$\text{AlignmentScore} = \mathbf{Q} \ \mathbf{K}^T \in \mathbb{R}^{N,N}$ 

    - Hint: Our 'query' and 'keys' are both matrix so you might want to use 'torch's matrix multiplication'.
    - Expected SHAPE : (bs, seq_len, seq_len) because we want to match each time step in self with each time step of itself
    
**5. Padding Mask**

Since there are many padding tokens in our input sequence. It would be inefficient to leave them as is. Please implement 'pad_mask' which will replace the Alignment Scores with -1e9 where the input sequence is the padding token.

**Skipping this step will not affect the next parts :)

**6. Calculate Attention Weights :**

    - Pass the Alignment Scores through Softmax
    
$\text{Attention Weights} = \text{softmax}(\text{AlignmentScore}) \in \mathbb{R}^{N,N} $ 

**7. Calculate the Context Vector :**

    - Multiply the Attention Weights with the 'values' to get the Context vector of SHAPE : (bs, seq_len, hidden_dim * num_directions)
    - Then do 'Sequence Length Reduction' to aggregate the dimension of seq_len into 1.
    - You can choose between averaging or sum.
    - Finally, Context vector should have SHAPE : (bs, hidden_dim * num_directions)
    
$\text{Context Vector} = \text{Attention Weights} \ \mathbf{V} \in \mathbb{R}^{h} $ 
    
    
**8. Finally, we use this Context Vector as the output of our Attention Mechanism**

### LSTM + Self Attention

In [9]:
# This attention mask will be apply after Q @ K^T thus the shape will be batch, seq_len, seq_len
def get_pad_mask(text):  #[batch, seq_len]
    batch_size, seq_len = text.size()
    # eq(zero) is lstm output over PAD token
    pad_mask = text.data.eq(0).unsqueeze(1)  # torch.eq Computes element-wise equality # batch_size x 1 x seq_len; we unsqueeze so we can make expansion below
    return pad_mask.expand(batch_size, seq_len, seq_len)  # batch_size x seq_len x seq_len

class LSTM_SelfAtt(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, output_dim, len_reduction):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, embed_dim, padding_idx=pad_idx)
        
        # let's use pytorch's LSTM
        self.lstm = nn.LSTM(embed_dim, 
                           hidden_dim, 
                           num_layers=num_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        self.softmax       = nn.LogSoftmax(dim=1)

        self.lin_Q = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.lin_K = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.lin_V = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        
        self.len_reduction = len_reduction
        
        # Linear Layer for binary classification 
        self.fc    = nn.Linear(hidden_dim * 2, output_dim)
        
    def self_attention_net(self, lstm_output, pad_mask):
        
        Q = self.lin_Q(torch.clone(lstm_output)) # SHAPE : (bs, seq_len, n_hidden * num_directions)
        K = self.lin_K(torch.clone(lstm_output)) # SHAPE : (bs, seq_len, n_hidden * num_directions)
        V = self.lin_V(torch.clone(lstm_output)) # SHAPE : (bs, seq_len, n_hidden * num_directions)
                
        alignment_score = torch.matmul(Q, K.transpose(1, 2)) # SHAPE : (bs, seq_len, seq_len)
                
        # Apply padding mask 
        if self.mask:
            alignment_score.masked_fill_(pad_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        
        soft_attn_weights = self.softmax(alignment_score)
        
        context = torch.matmul(soft_attn_weights, V) # SHAPE : (bs, seq_len, hidden_dim * num_directions)
        
        if self.len_reduction == "mean":
            return torch.mean(context, dim=1)
        elif self.len_reduction == "sum":
            return torch.sum(context, dim=1)
        elif self.len_reduction == "last":
            return context[:, -1, :]
        
    def forward(self, text, text_lengths, mask=True):
        self.mask = mask
        pad_mask = get_pad_mask(text)
        
        embedded = self.embedding(text) # SHAPE : (batch_size, seq_len, embed_dim)

        lstm_output, (hn, cn) = self.lstm(embedded)
        
        # This is how we concatenate the forward hidden and backward hidden from Pytorch's BiLSTM
        hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1)

        attn_output = self.self_attention_net(lstm_output, pad_mask)
        
        return self.fc(attn_output)

#### Reference & Further Readings:

LSTM :

- https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://medium.com/@raghavaggarwal0089/bi-lstm-bc3d68da8bd0

Attention : 

- https://arxiv.org/pdf/1409.0473.pdf
- https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a
- https://machinelearningmastery.com/the-attention-mechanism-from-scratch/#:~:text=In%20essence%2C%20when%20the%20generalized,the%20others%20in%20the%20sequence.
- https://blog.floydhub.com/attention-mechanism/#bahdanau-att
- https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html
- https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/
